In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
y_label_df = pd.read_csv("./data/train_label.csv")

In [3]:
y_label_df.head()

,acc_id,label
0,b8fbf3f6a70e3f36843bffc70c18ff51a0d755a87616ec...,week
1,ed500c4957956b3e99dc3985666850b582f812405eefb6...,week
2,acc6afa23a6bf15e18151e4794c7789225ef9d682f473c...,week
3,34095a3c9a2937ced3ea3fd75e22ce177dc5879d2a53f7...,week
4,26f3db6e8817a93c4ceda9a16f0832945e43d950b95882...,week


In [4]:
set(y_label_df["label"].tolist())

{'2month', 'month', 'retained', 'week'}

In [5]:
y = pd.Series([1,1,0,1], index=["2month", "month", "retained","week"])

In [6]:
y_label_df.head()

,acc_id,label
0,b8fbf3f6a70e3f36843bffc70c18ff51a0d755a87616ec...,week
1,ed500c4957956b3e99dc3985666850b582f812405eefb6...,week
2,acc6afa23a6bf15e18151e4794c7789225ef9d682f473c...,week
3,34095a3c9a2937ced3ea3fd75e22ce177dc5879d2a53f7...,week
4,26f3db6e8817a93c4ceda9a16f0832945e43d950b95882...,week


In [7]:
y

2month      1
month       1
retained    0
week        1
dtype: int64

In [8]:
y_label_df["class"] = y_label_df["label"].map(y)

In [9]:
sum(y_label_df["class"])

75000

In [10]:
X_raw_df = pd.read_csv("./data/train_activity.csv")

In [12]:
X_df = X_raw_df.groupby(["acc_id"]).mean().reset_index()

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_payment_df = pd.read_csv("./data/train_payment.csv")
X_df = X_df.merge(X_payment_df.groupby("acc_id").sum().reset_index(), how="left", on="acc_id")

In [15]:
len(X_df)

100000

In [16]:
X_df = X_df.fillna(0)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X_df.values, y_label_df["class"].values, test_size=0.2, stratify=y_label_df["class"])

In [18]:
sum(y_train == 1), sum(y_train == 0) 

(60000, 20000)

In [19]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.neural_network import MLPClassifier

In [20]:
rfc = RandomForestClassifier()

In [21]:
rfc.fit(X_train[:, 1:], y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [22]:
rfc.feature_importances_

array([0.01932354, 0.0434554 , 0.31354456, 0.03412596, 0.03772508,
       0.02153164, 0.03520538, 0.02430677, 0.11087226, 0.03939825,
       0.00418795, 0.00377545, 0.00459355, 0.00395142, 0.01848002,
       0.02002067, 0.00342   , 0.02270145, 0.00519944, 0.00625603,
       0.00099148, 0.01668722, 0.01869638, 0.00349148, 0.02074565,
       0.00439612, 0.00532776, 0.00104793, 0.01944865, 0.01826979,
       0.006284  , 0.02944589, 0.01044521, 0.00083233, 0.04490114,
       0.00094596, 0.01290229, 0.00559668, 0.00746922])

In [23]:
from sklearn.metrics import accuracy_score

In [24]:
y_pred = rfc.predict(X_test[:,1:])

In [25]:
accuracy_score(y_test, y_pred)

0.67055

In [27]:
sum(rfc.predict(X_df.values[:,1:]) == y_label_df["class"].values) / len(y_label_df)

0.90325